In [1]:
import pandas as pd
from mlxtend.frequent_patterns import apriori, association_rules

# Step 1: Load and preprocess data
df = pd.read_csv("MBA.csv", header=None)
transactions = df.apply(lambda row: row.dropna().tolist(), axis=1).tolist()

# Convert to one-hot encoded DataFrame
from mlxtend.preprocessing import TransactionEncoder
te = TransactionEncoder()
te_ary = te.fit(transactions).transform(transactions)
df_encoded = pd.DataFrame(te_ary, columns=te.columns_)

# Step 2: Generate frequent itemsets and association rules
frequent_itemsets = apriori(df_encoded, min_support=0.01, use_colnames=True)
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)

# Step 3: Recommendation system function
def recommend_items(user_items, rules_df, top_n=5):
    user_items = set(user_items)
    filtered_rules = rules_df[rules_df['antecedents'].apply(lambda x: x.issubset(user_items))]
    filtered_rules = filtered_rules.sort_values(by=['confidence', 'lift'], ascending=False)

    recommendations = []
    for consequents in filtered_rules['consequents']:
        for item in consequents:
            if item not in user_items:
                recommendations.append(item)
    return list(dict.fromkeys(recommendations))[:top_n]

# Step 4: Terminal interaction
def main():
    print("\n🛒 Welcome to the Market Basket Recommender!")
    print("Type the items in your basket, separated by commas.")
    user_input = input("Enter items: ").lower().strip()
    user_items = [item.strip() for item in user_input.split(',') if item.strip()]

    recommendations = recommend_items(user_items, rules)

    print("\n📦 Recommended Items Based on Your Basket:")
    if recommendations:
        for i, rec in enumerate(recommendations, 1):
            print(f"{i}. {rec}")
    else:
        print("❌ No strong recommendations found for the items you entered.")
        print("\n🔎 Try entering more common items like:")
        top_items = frequent_itemsets.sort_values(by="support", ascending=False)['itemsets'].head(10)
        for itemset in top_items:
            print("•", ", ".join(itemset))

if __name__ == "__main__":
    main()



🛒 Welcome to the Market Basket Recommender!
Type the items in your basket, separated by commas.

📦 Recommended Items Based on Your Basket:
1. mineral water
2. spaghetti
3. french fries
4. chocolate
5. milk
